# 📥 Load CSV to Delta Tables - Customer 360 + Marketing Campaigns

Ce notebook charge tous les fichiers CSV en tables Delta dans le Lakehouse.

## Tables créées :
- **CRM (6 tables)** : accounts, customers, segments, customer_segments, interactions, customer_profile
- **Marketing (5 tables)** : campaigns, assets, audiences, sends, events
- **Commerce (4 tables)** : products, orders, order_lines, returns

**Total : 15 tables Delta**

In [ ]:
# Cell 1: Configuration et vérification
print("📊 Marketing 360 - CSV to Delta Tables")
print("=" * 50)

from pyspark.sql.functions import col, to_timestamp, to_date

# Vérifier que Spark est disponible
print(f"✅ Spark version: {spark.version}")
print(f"✅ Lakehouse connecté")

In [ ]:
# Cell 2: Vérifier la structure des fichiers dans le Lakehouse

print("🔍 Vérification de la structure des fichiers...\n")

# Lister les fichiers dans Files pour trouver où sont les CSV
try:
    # Essayer de lister Files/raw/crm
    files_crm = mssparkutils.fs.ls("Files/crm")
    print("✅ Dossier trouvé: Files/crm/")
    print(f"   Fichiers: {len(files_crm)}")
    base_path = "Files/"
except:
    try:
        # Essayer Files/crm
        files_crm = mssparkutils.fs.ls("Files/crm")
        print("✅ Dossier trouvé: Files/crm/")
        print(f"   Fichiers: {len(files_crm)}")
        base_path = "Files"
    except:
        print("❌ ERREUR: Dossier CRM non trouvé!")
        print("📌 Vérifiez que vous avez bien uploadé les dossiers crm/, marketing/, commerce/ dans Files/")
        print("\nStructure attendue:")
        print("Files/")
        print("  ├── crm/")
        print("  ├── marketing/")
        print("  └── commerce/")
        raise

print(f"\n📁 Chemin de base détecté: {base_path}/")
print(f"✅ Les fichiers CSV seront chargés depuis ce chemin\n")

In [ ]:
# Cell 3: Définir les chemins des fichiers CSV

print("📁 Configuration des chemins des fichiers...\n")

# Utiliser le base_path détecté
# Fichiers CRM
crm_files = {
    "crm_accounts": f"{base_path}/crm/crm_accounts.csv",
    "crm_customers": f"{base_path}/crm/crm_customers.csv",
    "crm_segments": f"{base_path}/crm/crm_segments.csv",
    "crm_customer_segments": f"{base_path}/crm/crm_customer_segments.csv",
    "crm_interactions": f"{base_path}/crm/crm_interactions.csv",
    "crm_customer_profile": f"{base_path}/crm/crm_customer_profile.csv"
}

# Fichiers Marketing
marketing_files = {
    "marketing_campaigns": f"{base_path}/marketing/marketing_campaigns.csv",
    "marketing_assets": f"{base_path}/marketing/marketing_assets.csv",
    "marketing_audiences": f"{base_path}/marketing/marketing_audiences.csv",
    "marketing_sends": f"{base_path}/marketing/marketing_sends.csv",
    "marketing_events": f"{base_path}/marketing/marketing_events.csv"
}

# Fichiers Commerce
commerce_files = {
    "products": f"{base_path}/commerce/products.csv",
    "orders": f"{base_path}/commerce/orders.csv",
    "order_lines": f"{base_path}/commerce/order_lines.csv",
    "returns": f"{base_path}/commerce/returns.csv"
}

# Fusionner tous les fichiers
all_files = {**crm_files, **marketing_files, **commerce_files}

print(f"📋 CRM tables: {len(crm_files)}")
print(f"📋 Marketing tables: {len(marketing_files)}")
print(f"📋 Commerce tables: {len(commerce_files)}")
print(f"📋 Total tables: {len(all_files)}\n")

In [ ]:
# Cell 4: Charger les tables CRM

print("🔷 Chargement des tables CRM...\n")

for table_name, file_path in crm_files.items():
    print(f"  Loading {table_name}...", end=" ")
    
    # Lire le CSV
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    
    # Écrire en table Delta
    df.write.format("delta").mode("overwrite").saveAsTable(table_name)
    
    row_count = df.count()
    print(f"✅ {row_count:,} lignes")

print("\n✅ Tables CRM créées avec succès!")

In [ ]:
# Cell 5: Charger les tables Marketing

print("📧 Chargement des tables Marketing...\n")

for table_name, file_path in marketing_files.items():
    print(f"  Loading {table_name}...", end=" ")
    
    # Lire le CSV
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    
    # Écrire en table Delta
    df.write.format("delta").mode("overwrite").saveAsTable(table_name)
    
    row_count = df.count()
    print(f"✅ {row_count:,} lignes")

print("\n✅ Tables Marketing créées avec succès!")

In [ ]:
# Cell 6: Charger les tables Commerce

print("🛒 Chargement des tables Commerce...\n")

for table_name, file_path in commerce_files.items():
    print(f"  Loading {table_name}...", end=" ")
    
    # Lire le CSV
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    
    # Écrire en table Delta
    df.write.format("delta").mode("overwrite").saveAsTable(table_name)
    
    row_count = df.count()
    print(f"✅ {row_count:,} lignes")

print("\n✅ Tables Commerce créées avec succès!")

In [ ]:
# Cell 7: Résumé et validation

print("\n" + "=" * 60)
print("🎉 RÉSUMÉ - TABLES DELTA CRÉÉES")
print("=" * 60 + "\n")

tables_created = []

print("📊 CRM Tables:")
for table_name in crm_files.keys():
    count = spark.table(table_name).count()
    print(f"  ✅ {table_name}: {count:,} lignes")
    tables_created.append(table_name)

print("\n📧 Marketing Tables:")
for table_name in marketing_files.keys():
    count = spark.table(table_name).count()
    print(f"  ✅ {table_name}: {count:,} lignes")
    tables_created.append(table_name)

print("\n🛒 Commerce Tables:")
for table_name in commerce_files.keys():
    count = spark.table(table_name).count()
    print(f"  ✅ {table_name}: {count:,} lignes")
    tables_created.append(table_name)

print(f"\n📌 Total: {len(tables_created)} tables Delta créées")
print("\n📍 Les tables sont maintenant disponibles dans la section 'Tables' du Lakehouse")
print("📍 Prochaine étape: Créer le Semantic Model à partir de ces tables")

In [ ]:
# Cell 7: Aperçu des données (optionnel)

print("👀 Aperçu des données:\n")

print("=== CRM CUSTOMERS (5 premières lignes) ===")
spark.sql("SELECT * FROM crm_customers LIMIT 5").show()

print("\n=== MARKETING CAMPAIGNS (5 premières lignes) ===")
spark.sql("SELECT campaign_id, campaign_name, objective, status, start_date FROM marketing_campaigns LIMIT 5").show()

print("\n=== ORDERS (5 premières lignes) ===")
spark.sql("SELECT order_id, customer_id, order_date, total_amount_eur, status FROM orders LIMIT 5").show()